In [1]:
import numpy as np
import optuna
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score

c:\Users\AlessandroKuz\.conda\envs\MachineLearningITS\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = load_iris()
X, y = data.data, data.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=21)

In [4]:
preprocessing_pipe = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5, weights='distance')),
    ('scaler', RobustScaler()),
    ('poly', PolynomialFeatures())
])

In [5]:
X_train = preprocessing_pipe.fit_transform(X_train)
X_test = preprocessing_pipe.transform(X_test)

In [6]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

def my_objective(trial):

    params = {
        'objective': 'multi:softmax',
        'num_class': 3,
        'eval_metric': 'mlogloss',
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'eta': trial.suggest_float('eta', 1e-8, 1.0, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-8, 1.0, log=True),
        'max_delta_step': trial.suggest_float('max_delta_step', 1e-8, 1.0, log=True),
    }

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'test-mlogloss')

    skf_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=21)

    cv_results = xgb.cv(params, dtrain, 
                        num_boost_round=100, 
                        folds=skf_cv, 
                        metrics='mlogloss', 
                        early_stopping_rounds=10, seed=21, 
                        callbacks=[pruning_callback]
                        )
    
    clf = xgb.train(params, dtrain, num_boost_round=cv_results.shape[0])
    y_pred = clf.predict(dtest)

    trial.set_user_attr('accuracy', accuracy_score(y_test, y_pred))
    trial.set_user_attr('f1', f1_score(y_test, y_pred, average='macro'))
    trial.set_user_attr('cohen_kappa', cohen_kappa_score(y_test, y_pred))
    trial.set_user_attr('n_estimators', len(cv_results))

    result = cv_results['test-mlogloss-mean'].values[-1]
    return result

study = optuna.create_study(direction='minimize', 
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5, interval_steps=2), 
                            study_name='xgboost_iris', storage='sqlite:///xgboost_iris.db', load_if_exists=True)

study.optimize(my_objective, n_trials=100, n_jobs=1)

print(study.best_params)
print(study.best_value)

[I 2024-03-26 11:40:39,388] Using an existing study with name 'xgboost_iris' instead of creating a new one.
[I 2024-03-26 11:40:40,790] Trial 165 finished with value: 1.098612276713053 and parameters: {'alpha': 1.0474383148973715e-06, 'colsample_bylevel': 0.7498765197211659, 'colsample_bytree': 0.9666447766880003, 'eta': 0.21384522781522025, 'gamma': 0.7532188880590253, 'grow_policy': 'depthwise', 'lambda': 8.631761143436675e-06, 'max_delta_step': 0.0001349432093513624, 'max_depth': 1, 'min_child_weight': 0.4815819180543842, 'subsample': 0.9347536565301062}. Best is trial 159 with value: 0.13726988235333315.
[I 2024-03-26 11:40:42,037] Trial 166 finished with value: 1.0986122528711957 and parameters: {'alpha': 2.1034798035522523e-06, 'colsample_bylevel': 0.7975020333704823, 'colsample_bytree': 0.9241503073075399, 'eta': 0.39222813578218746, 'gamma': 0.30753851793841785, 'grow_policy': 'depthwise', 'lambda': 3.9215420825719733e-05, 'max_delta_step': 1.1485164349731775e-06, 'max_depth': 

{'alpha': 1.0348552680240616e-07, 'colsample_bylevel': 0.7119533307124649, 'colsample_bytree': 0.7855125009981918, 'eta': 0.7695360913800767, 'gamma': 4.400021596962805e-06, 'grow_policy': 'depthwise', 'lambda': 2.405641881129051e-05, 'max_delta_step': 0.6695580669629951, 'max_depth': 1, 'min_child_weight': 0.041086014846130364, 'subsample': 0.6688679659736075}
0.11077952127670869


In [7]:
xgb.XGBClassifier(**study.best_params).fit(X_train, y_train).score(X_test, y_test)

0.9333333333333333

In [8]:
df = study.trials_dataframe()
df.loc[df['state']=='COMPLETE', ['user_attrs_accuracy', 'user_attrs_f1', 'user_attrs_cohen_kappa', 'value']].sort_values('value')

,user_attrs_accuracy,user_attrs_f1,user_attrs_cohen_kappa,value
260,0.933333,0.933333,0.9,0.110780
257,0.933333,0.933333,0.9,0.110940
248,0.933333,0.933333,0.9,0.111109
243,0.933333,0.933333,0.9,0.112181
242,0.933333,0.933333,0.9,0.112295
...,...,...,...,...
107,0.333333,0.166667,0.0,1.098612
117,0.333333,0.166667,0.0,1.098612
4,0.333333,0.166667,0.0,1.098612
1,0.333333,0.166667,0.0,1.098612
